In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score

In [59]:
hp_df=pd.read_csv('House Price Data.csv')
y=hp_df['SalePrice']

#Droping output variable from dataset

hp_df.drop(columns=['SalePrice','Id'],inplace=True)

nadf_info=pd.DataFrame({'col_name':hp_df.columns,'na_cnt':hp_df.isnull().sum(),'na_pct':(hp_df.isnull().sum()/hp_df.shape[0])*100})

cols_grt50_ls=list(nadf_info[nadf_info['na_pct']>50]['col_name'])

hp_df.drop(columns=cols_grt50_ls,inplace=True)

x_train,x_test,y_train,y_test=train_test_split(hp_df,y,random_state=49,test_size=0.2)

con_cols=[col for col in hp_df.columns if hp_df[col].dtype=='int64' or hp_df[col].dtype=='float64']

cat_cols=[col for col in hp_df.columns if hp_df[col].dtype=='object']

## filling missing values for contnous columns 
for col in con_cols:
    x_train[col].fillna(x_train[col].mean(),inplace=True)
    x_test[col].fillna(x_train[col].mean(),inplace=True)

## filling missing values for catgorical columns 
    
for col in cat_cols:
    x_train[col].fillna(x_train[col].mode()[0],inplace=True)
    x_test[col].fillna(x_train[col].mode()[0],inplace=True)


In [60]:
scalar=StandardScaler()

for col in con_cols:
    x_train[col]=scalar.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scalar.transform(np.array(x_test[col]).reshape(-1,1))


In [61]:
cat_encd_train= pd.get_dummies(x_train[cat_cols])
cat_encd_test=pd.get_dummies(x_test[cat_cols])

In [62]:
cat_encd_train_final,cat_encd_test_final=cat_encd_train.align(cat_encd_test,join='inner',axis=1)
cat_encd_train_final.shape###aligning train & test data one hot encoded catg columns due to unqual no of columns i.e no of cilumns would differ for that we align to get same

(1168, 197)

In [63]:
x_train_final=pd.concat([x_train[con_cols],cat_encd_train_final],axis=1)


In [64]:
x_test_final=pd.concat([x_test[con_cols],cat_encd_test_final],axis=1)

In [65]:
lin_reg=LinearRegression()
lin_reg.fit(x_train_final,y_train)

LinearRegression()

In [66]:
y_test_pred=lin_reg.predict(x_test_final)

In [67]:
lin_reg.score(x_test_final,y_test)

0.8755431979271163

In [68]:
lin_reg.score(x_train_final,y_train)

0.8977182664016318

In [69]:
r2_score(y_test,y_test_pred)

0.8755431979271163